In [1]:
!pip install flash-attn --no-build-isolation
!pip install triton

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 56.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp310-cp310-linux_x86_64.whl size=187797312 sha256=b267f80a08e516292cdd748056a2178a45b8abedf7fca123292eb17c21c8c87c
  Stored in directory: /root/.cache/pip/wheels/59/ce/d5/08ea07bfc16ba218dc65a3a7ef9b6a270530bcbd2cea2ee1ca
Successfully built flash-attn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:000:00:0100:01


In [5]:
import torch
from torch import Tensor,nn
from typing import Dict, List, Optional, Union

import dataclasses


@dataclasses.dataclass
class TensorPointer:
    """Dataclass specifying from which rank we need to query a tensor from in order to access data"""

    # Needed to understand from which rank to get the tensor
    # TODO @thomasw21: Maybe add which group it belongs to as well? Typically this is highly correlated to `p2p.pg`
    group_rank: int
    # TODO @thomasw21: Maybe add a tag (torch.distributed.send/recv allow for tagging)

class TritonRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5, device=None, dtype=None):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(hidden_size, device=device, dtype=dtype))
        self.register_parameter("bias", None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.ones_(self.weight)

    def forward(
        self, input, residual=None, dropout_p=0.0, prenorm=False, residual_in_fp32=False, return_dropout_mask=False
    ):
        from flash_attn.ops.triton.layer_norm import layer_norm_fn

        return layer_norm_fn(
            input,
            self.weight,
            None,
            residual=residual,
            eps=self.eps,
            dropout_p=dropout_p,
            prenorm=prenorm,
            residual_in_fp32=residual_in_fp32,
            is_rms_norm=True,
            return_dropout_mask=return_dropout_mask,
        )

class RotaryEmbedding(nn.Module):
    def __init__(self, dim: int, end: int, theta: float = 10000.0):
        super().__init__()
        assert dim % 2 == 0
        self.dim = dim
        self.end = end
        self.theta = theta
        self.register_buffer(
            "freqs_cis",
            self._compute_freqs(end, dim, theta),
            persistent=False,
        )

    def _compute_freqs(self, end, dim, theta):
        freqs = 1.0 / (theta ** (torch.arange(0, dim, 2, dtype=torch.float) / dim))
        t = torch.arange(end)
        freqs = torch.outer(t, freqs)
        complex_freqs = torch.polar(torch.ones_like(freqs), freqs)
        return torch.view_as_real(complex_freqs)

    def forward(self, x: torch.Tensor, position_ids: Optional[torch.LongTensor]):
        batch_size, seq_length, num_heads, inner_dim = x.shape
        dtype = x.dtype
        x = x.view(batch_size, seq_length, num_heads, inner_dim // 2, 2)
        complex_x = torch.view_as_complex(x.float())

        if position_ids is None:
            freqs_cis = self.freqs_cis[:seq_length]
        else:
            freqs_cis = self.freqs_cis[position_ids]

        complex_freqs = torch.view_as_complex(freqs_cis)
        x_out = torch.view_as_real(complex_x * complex_freqs).view(batch_size, seq_length, num_heads, inner_dim)
        return x_out.type(dtype)

class CoreAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.hidden_size % config.num_attention_heads == 0, "Hidden size must be divisible by number of attention heads."
        self.d_qk = config.hidden_size // config.num_attention_heads
        self.d_v = config.hidden_size // config.num_attention_heads
        #self.is_using_mup = config.is_using_mup

    def forward(self, query_states, key_states, value_states, q_sequence_mask, kv_sequence_mask):
        cu_seqlens_q = torch.cumsum(q_sequence_mask.sum(-1, dtype=torch.int32), dim=0, dtype=torch.int32)
        cu_seqlens_k = torch.cumsum(kv_sequence_mask.sum(-1, dtype=torch.int32), dim=0, dtype=torch.int32)
        causal = q_sequence_mask.shape[1] != 1
        #softmax_scale = 1 / query_states.shape[-1] if self.is_using_mup else None

        attn_output = torch.nn.functional.scaled_dot_product_attention(
            query_states, key_states, value_states, attn_mask=None, dropout_p=0.0
        )
        return attn_output

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        from flash_attn.layers.rotary import RotaryEmbedding as FlashRotaryEmbedding
        
        self.n_q_heads = config.num_attention_heads
        self.n_kv_heads = config.num_key_value_heads if hasattr(config, 'num_key_value_heads') else config.num_attention_heads
        self.n_repeats = config.num_attention_heads // self.n_kv_heads
        self.is_gqa = config.num_attention_heads != self.n_kv_heads
        self.d_qk = config.hidden_size // config.num_attention_heads
        self.d_v = config.hidden_size // config.num_attention_heads
        self.d_model = config.hidden_size

        self.qkv_proj = nn.Linear(self.d_model, config.num_attention_heads * self.d_qk + 2 * self.n_kv_heads * self.d_qk, bias=False)
        self.rotary_embedding = RotaryEmbedding(dim=self.d_qk, end=config.max_position_embeddings, theta=config.rope_theta)
        self.flash_rotary_embedding = FlashRotaryEmbedding(dim=self.d_qk, base=config.rope_theta, interleaved=config.rope_interleaved)
        self.o_proj = nn.Linear(config.num_attention_heads * self.d_qk, self.d_model, bias=False)
        self.attention = CoreAttention(config)

    def forward(self, hidden_states, sequence_mask):
        qkv_states = self.qkv_proj(hidden_states)
        q_length, batch_size, _ = qkv_states.shape

        if self.is_gqa:
            query_states, key_states, value_states = torch.split(
                qkv_states,
                [self.n_q_heads * self.d_qk, self.n_kv_heads * self.d_qk, self.n_kv_heads * self.d_qk],
                dim=-1,
            )
        else:
            query_states, key_states, value_states = qkv_states.view(q_length, batch_size, 3, self.n_q_heads, self.d_qk).permute(2, 1, 0, 3, 4).contiguous()

        query_states, key_value_states = self.flash_rotary_embedding(query_states, kv=torch.stack([key_states, value_states], dim=2))
        key_states, value_states = torch.chunk(key_value_states, 2, dim=2)

        attention_output = self.attention(
            query_states=query_states.view(batch_size * q_length, self.n_q_heads, self.d_qk),
            key_states=key_states.view(batch_size * q_length, self.n_kv_heads, self.d_qk),
            value_states=value_states.view(batch_size * q_length, self.n_kv_heads, self.d_v),
            q_sequence_mask=sequence_mask,
            kv_sequence_mask=sequence_mask,
        )

        attention_output = attention_output.contiguous().view(batch_size, q_length, self.n_q_heads * self.d_v).transpose(0, 1)
        output = self.o_proj(attention_output)

        return {"hidden_states": output, "sequence_mask": sequence_mask}

class ColumnLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
        super().__init__(in_features, out_features, bias, device, dtype)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #x = x.detach()  # Detach from computation graph
        return nn.functional.linear(x, self.weight, self.bias)


class RowLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
        super().__init__(in_features, out_features, bias, device, dtype)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #x = x.detach()  # Detach from computation graph
        return nn.functional.linear(x, self.weight, self.bias)

class GELUActivation(nn.Module):
    """
    Original Implementation of the GELU activation function in Google BERT repo when initially created. For
    information: OpenAI GPT's GELU is slightly different (and gives slightly different results): 0.5 * x * (1 +
    torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3)))) This is now written in C in nn.functional
    Also see the Gaussian Error Linear Units paper: https://arxiv.org/abs/1606.08415
    """

    def __init__(self, use_gelu_python: bool = False):
        super().__init__()
        if use_gelu_python:
            self.act = self._gelu_python
        else:
            self.act = nn.functional.gelu

    def _gelu_python(self, input: Tensor) -> Tensor:
        return input * 0.5 * (1.0 + torch.erf(input / math.sqrt(2.0)))

    def forward(self, input: Tensor) -> Tensor:
        return self.act(input)

class GLUActivation(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.functional.silu

    def forward(self, merged_states: torch.Tensor):
        gate_states, up_states = torch.split(merged_states, merged_states.shape[-1] // 2, dim=-1)
        return self.act(gate_states) * up_states


class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.gate_up_proj = ColumnLinear(config.hidden_size, 2*config.intermediate_size, bias=False)
        self.down_proj = RowLinear(config.intermediate_size, config.hidden_size, bias=False)
        self.split_silu_mul = GLUActivation()

    def forward(self, hidden_states):
        merged_states = self.gate_up_proj(hidden_states)
        hidden_states = self.down_proj(self.split_silu_mul(merged_states))
        return {"hidden_states": hidden_states}


class LlamaDecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.input_layernorm = TritonRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.attn = CausalSelfAttention(config)
        self.post_attention_layernorm = TritonRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.mlp = MLP(config)

    def _core_forward(
        self,
        hidden_states: Union[torch.Tensor, TensorPointer],
        sequence_mask: Union[torch.Tensor, TensorPointer],
    ) -> List[Union[torch.Tensor, TensorPointer]]:
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)

        output = self.attn(hidden_states=hidden_states, sequence_mask=sequence_mask)
        hidden_states = output["hidden_states"]
        hidden_states = hidden_states + residual

        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = self.mlp(hidden_states=hidden_states)["hidden_states"]
        hidden_states = hidden_states + residual

        return hidden_states, output["sequence_mask"]

    def forward(
        self,
        hidden_states: Union[torch.Tensor, TensorPointer],
        sequence_mask: Union[torch.Tensor, TensorPointer],
    ) -> Dict[str, Union[torch.Tensor, TensorPointer]]:


        hidden_states, sequence_mask = self._core_forward(hidden_states, sequence_mask)

        return {
            "hidden_states": hidden_states,
            "sequence_mask": sequence_mask,
        }

class LlamaModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        #layer_idx = config.num_layers
        self.layers = nn.ModuleList([LlamaDecoderLayer(config) for _ in range(config.num_layers)])
        self.final_layer_norm = TritonRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.lm_head = ColumnLinear(config.hidden_size, config.vocab_size, bias=False)
        self.cast_to_fp32 = lambda: lambda x: x.float()

    def forward(
        self,
        input_ids: Union[torch.Tensor, TensorPointer],  # [batch_size, seq_length]
        input_mask: Union[torch.Tensor, TensorPointer],  # [batch_size, seq_length]
    ):
        return self.forward_with_hidden_states(input_ids=input_ids, input_mask=input_mask)[0]

    def forward_with_hidden_states(
        self,
        input_ids: Union[torch.Tensor, TensorPointer],  # [batch_size, seq_length]
        input_mask: Union[torch.Tensor, TensorPointer],  # [batch_size, seq_length]
    ):
        # Format input in `[seq_length, batch_size]` to support high TP with low batch_size
        input_ids = input_ids.transpose(0, 1)
        hidden_states = self.embed_tokens(input_ids)

        hidden_encoder_states = {
            "hidden_states": hidden_states,
            "sequence_mask": input_mask,
        }

        for encoder_block in self.layers:
            hidden_encoder_states = encoder_block(**hidden_encoder_states)

        hidden_encoder_states["hidden_states"] = self.final_layer_norm(hidden_encoder_states["hidden_states"])

        sharded_logits = self.lm_head(x=hidden_encoder_states["hidden_states"])

        fp32_sharded_logits = sharded_logits.float()

        return fp32_sharded_logits, hidden_states

class CrossEntropyLossFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, logits, target):
        logits = logits - torch.max(logits, dim=-1, keepdim=True)[0]
        exp_logits = torch.exp(logits)
        sum_exp_logits = exp_logits.sum(dim=-1, keepdim=True)
        log_probs = logits - torch.log(sum_exp_logits)

        loss = -log_probs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1)
        ctx.save_for_backward(exp_logits / sum_exp_logits, target)
        return loss

    @staticmethod
    def backward(ctx, grad_output):
        softmax, target = ctx.saved_tensors
        grad_input = softmax.clone()
        grad_input.scatter_add_(dim=-1, index=target.unsqueeze(-1), src=-1.0)
        grad_input.mul_(grad_output.unsqueeze(-1))
        return grad_input, None


def cross_entropy_loss(logits, target, dtype: torch.dtype = None):
    if dtype is not None:
        logits = logits.to(dtype=dtype)
    return CrossEntropyLossFunction.apply(logits, target)


class Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, logits: torch.Tensor, label_ids: torch.Tensor, label_mask: torch.Tensor) -> dict:
        loss = cross_entropy_loss(logits, label_ids.transpose(0, 1).contiguous(), dtype=torch.float).transpose(0, 1)
        loss = (loss * label_mask).sum() / label_mask.sum()
        return {"loss": loss}


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import math
import os
from numpy import arange
import torch.nn.functional as F

# Import the LlamaModel from model_manual.py
#from model_manual import LlamaModel

# Function to generate text from the model
def generate_text(model, input_text, vocab, id_to_token, device, max_length=50, temperature=0.7):
    model.eval()
    input_ids = torch.tensor([[vocab.get(token, vocab['<|endoftext|>']) for token in input_text.split()]], dtype=torch.long).to(device)
    generated_tokens = input_ids.tolist()[0]

    with torch.no_grad():
        for _ in range(max_length):
            logits = model(input_ids)[:, -1, :]
            logits = logits / temperature
            probabilities = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probabilities, num_samples=1).squeeze().item()

            if next_token == vocab['<|endoftext|>']:
                break

            generated_tokens.append(next_token)
            input_ids = torch.tensor([generated_tokens], dtype=torch.long).to(device)

    return ' '.join(id_to_token[token] for token in generated_tokens if token in id_to_token)


class DataloaderLite:
    def __init__(self, file_path, seq_len, batch_size):
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/cosmo2-tokenizer", add_prefix_space=True)
        if self.tokenizer.pad_token is None:
            if self.tokenizer.eos_token:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            else:
                self.tokenizer.add_special_tokens({"pad_token": "[PAD]"})
                self.tokenizer.resize_token_embeddings(len(self.tokenizer))

        with open(file_path, 'r', encoding='utf-8') as f:
            self.text = f.read()
        self.epochs = len(self.text) // (self.seq_len * self.batch_size)
        self.current_position = 0
        self.padded_chunks = []
        self.max_len = 0
        
    def get_max_length(self):
        return self.max_len

    def next_batch(self):

        self.chunks = [self.text[(self.current_position + i):(self.current_position + i + self.seq_len)] for i in range(0, self.seq_len*self.batch_size, self.seq_len)]
        self.current_position = self.current_position + self.seq_len*self.batch_size
        if self.current_position + (self.seq_len*self.batch_size + 1) > len(self.text):
            self.current_position = 0
        self.encoded_chunks = [self.tokenizer(chunk, return_tensors='pt', truncation=True, max_length=self.seq_len) for chunk in self.chunks]

        self.max_len = max(chunk['input_ids'].shape[1] for chunk in self.encoded_chunks)
        self.padded_chunks = []
        for chunk in self.encoded_chunks:
            input_ids = torch.cat((chunk['input_ids'], torch.full((1, self.max_len - chunk['input_ids'].shape[1]), self.tokenizer.pad_token_id, dtype=torch.long)), dim=1)
            attention_mask = torch.cat((chunk['attention_mask'], torch.zeros((1, self.max_len - chunk['attention_mask'].shape[1]), dtype=torch.long)), dim=1)
            self.padded_chunks.append(input_ids.squeeze(0))
            self.padded_chunks.append(attention_mask.squeeze(0))
        return self.padded_chunks

def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    attention_masks = torch.stack([item[1] for item in batch])
    return input_ids, attention_masks


# Random initialization
def init_weights(m):
    if isinstance(m, (torch.nn.Linear, torch.nn.Embedding)):
        torch.nn.init.xavier_uniform_(m.weight)
        if hasattr(m, 'bias') and m.bias is not None:
            torch.nn.init.zeros_(m.bias)

def print_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}\n")

    for name, param in model.named_parameters():
        print(f"{name}: {param.numel():,}")

def train_model(config, train_file, steps, output_dir):
    tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/cosmo2-tokenizer")

    if tokenizer.pad_token is None:
        if tokenizer.eos_token:
            tokenizer.pad_token = tokenizer.eos_token
        else:
            tokenizer.add_special_tokens({"pad_token": "[PAD]"})
            tokenizer.resize_token_embeddings(len(tokenizer))

    vocab = tokenizer.get_vocab()
    id_to_token = {v: k for k, v in vocab.items()}

    dataloader = DataloaderLite(train_file, SEQ_LEN, BATCH_SIZE)
    #padded_chunks = dataloader.next_batch()
    #print(padded_chunks[0])
    #print(padded_chunks[1])
    #print(padded_chunks[2])
    #print(padded_chunks[3])

    model = LlamaModel(config)
    model.apply(init_weights)
    for p in model.parameters():
        p.data.clamp_(-1e5, 1e5)
    print_model_parameters(model)
    model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    model.device = next(model.parameters()).device
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)
    
    #loss_fn = torch.nn.CrossEntropyLoss()
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    progress_bar = tqdm(range(steps), desc="Training")
    
    PROMPT = "inventory to particularise their abundance"
    
    step = 0
    while step < steps:
        input_tokens = dataloader.next_batch()
        max_len = dataloader.get_max_length()
        input_list = []
        attenttion_list = []
        for i in arange(BATCH_SIZE):
            input_list.append(input_tokens[2*i])
            attenttion_list.append(input_tokens[2*i+1])

        input_ids = torch.stack(input_list)
        inputs = input_ids[:, :-2]  # Keep batch dim and remove last token
        targets = input_ids[:, 1:-1]  # Keep batch dim and remove first token
        attention_mask = torch.stack(attenttion_list)
        attentions = attention_mask[:, :-2]
        device = next(model.parameters()).device
        inputs, attentions = inputs.to(device), attentions.to(device)
        #positions = torch.arange(0, inputs.size(1), dtype=torch.long).unsqueeze(0).repeat(inputs.size(0), 1).to(device)
        #print(inputs.shape)
        #print(attentions.shape)
        #print(positions.shape)
    
        optimizer.zero_grad()
        logits = model(inputs, attentions)
    
        labels = targets.to(device)  # Move labels to the same device as the model and inputs
        # Create a mask based on counts
        mask = attentions.bool()  # The attention mask already has the correct shape
        logits = logits.transpose(0, 1)  # Align logits to (batch_size, seq_len, vocab_size)
        logits_masked = logits[mask].contiguous().view(-1, config.vocab_size)
        labels_masked = labels[mask].contiguous().view(-1)
        probabilities = F.softmax(logits_masked, dim=-1)
        max_prob_indices = torch.argmax(probabilities, dim=-1)
        print(max_prob_indices[:10])
        print(labels_masked[:10])
        
        if mask.sum() == 0:
            raise ValueError("Attention mask sums to zero!")

        loss_cross_entropy = F.cross_entropy(logits_masked, labels_masked)
        #loss = loss_fn(logits_masked, labels_masked)
    
        loss_cross_entropy.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
    
        progress_bar.update(1)
        progress_bar.set_postfix(loss=loss_cross_entropy.item())
    
        #if step % 10 == 0:
        #    torch.save({
        #        'step': step,
        #        'model_state_dict': model.state_dict(),
        #        'optimizer_state_dict': optimizer.state_dict(),
        #        'scheduler_state_dict': scheduler.state_dict()
        #    }, os.path.join(output_dir, f'checkpoint_{step}.pt'))
        #    #generated_text = generate_text(model, PROMPT, vocab, id_to_token, model.device)
        #    #print(f"\nGenerated text at step {step}: {generated_text}\n")
    
        step += 1
        if step >= steps:
            break

class Config:
    pass

if __name__ == "__main__":
    config = Config()
    config.vocab_size = AutoTokenizer.from_pretrained("HuggingFaceTB/cosmo2-tokenizer").vocab_size
    config.num_layers = 30
    config.hidden_size = 576
    config.num_attention_heads = 8
    config.rms_norm_eps = 1.0e-05
    config.max_position_embeddings = 2048
    config.rope_theta = 500000.0
    config.hidden_act = False
    config.intermediate_size = 1536
    config.rope_interleaved = False
    #config.rope_scaling = null
    config.rope_theta = 10000.0

    BATCH_SIZE = 8
    SEQ_LEN = 512

    train_model(config, '/kaggle/input/assign13-era-v3-dataset/input.txt', 1000, './output')


tokenizer_config.json:   0%|          | 0.00/3.91k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/489 [00:00<?, ?B/s]

...

Training: 100%|█████████▉| 999/1000 [04:55<00:00,  3.59it/s, loss=4.8] 

tensor([ 28, 198,  42,  30, 260,  93,   0,  28, 198,  28], device='cuda:0')
tensor([  198,    54,  4985,   284,   430, 33659,  1718,    28,   441,   253],
       device='cuda:0')


Training: 100%|██████████| 1000/1000 [04:56<00:00,  3.59it/s, loss=4.82]

tensor([ 28, 198, 198, 198,  42,  42,  42,  42, 198, 198], device='cuda:0')
tensor([   17,   198,   198, 17321,  5819,  2154,  4501,    42,   198,    57],
       device='cuda:0')


Training: 100%|██████████| 1000/1000 [04:56<00:00,  3.38it/s, loss=4.82]
